In [1]:
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [2]:
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/data/Round2_train_data_without_append.zip"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="hc-02-1258788535"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="0622train_1.zip"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

####  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [13]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
import datetime, time 
from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
from zipfile import ZipFile
from collections import Counter
from sklearn.metrics import mean_squared_error as mse 
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [15]:
%%time

# myzip=ZipFile('data/0622train_1.zip')
# f=myzip.open('0622train_1.csv')
# train_df=pd.read_csv(f)
# print(train_df)
# f.close()
# myzip.close() 

# train_df.drop_duplicates(['loadingOrder', 'label'], inplace=True)

# del_cols = ['arrive_time_index', 'arrive_time', 'direction', 'geo_hash3', 'selcte_index']
# train_df.drop(del_cols, axis=1, inplace=True) 

# columns = ['TRANSPORT_TRACE', 'begin_port_name', 'begin_port_position',
#         'begin_port_position_hash3', 'begin_port_position_hash4',
#         'begin_port_position_hash5', 'carrierName', 'end_port_name',
#         'end_port_position', 'end_port_position_hash3',
#         'end_port_position_hash4', 'end_port_position_hash5', 'loadingOrder',
#         'vesselMMSI', 'test_index', 'geo_hash3', 'geo_hash5',
#         'geo_hash4', 'speed', 'longitude',
#         'latitude', 'timestamp', 'direction', 'label']
# train_df.columns = columns

test_df = pd.read_csv("data/Round2_test_create.csv")

CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48.5 ms


In [8]:
# %%time

# def get_stats(values):   # 统计特征
#     xlen = len(values)
#     vmax = np.max(values)
#     vmin = np.min(values)
#     return [np.mean(values), vmax, vmin, np.std(values), vmax-vmin,
#             np.sum(values), np.median(values), np.count_nonzero(values), len(set(values)), 
#             np.count_nonzero(values)/xlen, len(set(values))/xlen, 
#             # np.percentile(values, 25), np.percentile(values, 75), 
#             skew(values), kurtosis(values), 
#             sorted(Counter(values).items(), key=lambda x: x[1], reverse=True)[0][0]]

# def feature_engineering(df, is_train=True):
#     numerical_fea = []
#     categorical_fea = []
#     if is_train:
#         label = []
#     for idx in tqdm(range(df.shape[0])):
#         line = df.iloc[idx]

#         timestamp = [datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.000Z") for x in line['timestamp'].split(' ')]
        
#         # 时间特征
#         timestamp_hours = np.array([(x-timestamp[0]).total_seconds()/3600.0 for x in timestamp])  # 减去最初的时间
#         timestamp_diff1 = np.diff(timestamp_hours, prepend=timestamp_hours[0])    # 时间一阶差分
#         # print(timestamp_diff1[:20])
#         xlen = len(timestamp_hours)   # 长度

#         begin_port = [float(x) for x in line['begin_port_position'].split(' ')]
#         end_port = [float(x) for x in line['end_port_position'].split(' ')]

#         # 原始的四个特征
#         lon = [float(x) for x in line['longitude'].split(' ')]  # 经度
#         lat  = [float(x) for x in line['latitude'].split(' ')]  # 纬度
#         speed = [float(x) for x in line['speed'].split(' ')]
#         direction = [float(x) for x in line['direction'].split(' ')]
#         lon_diff1 = np.diff(lon, prepend=lon[0])
#         lat_diff1 = np.diff(lat, prepend=lat[0])
#         speed_diff1 = np.diff(speed, prepend=speed[0])
#         direction_diff1 = np.diff(direction, prepend=direction[0])

#         # 类别特征
#         carrierName = line['carrierName']
#         vesselMMSI = line['vesselMMSI']
#         begin_hash3, begin_hash4, begin_hash5 = line['begin_port_position_hash3'], line['begin_port_position_hash4'], line['begin_port_position_hash5']
#         end_hash3, end_hash4, end_hash5 = line['end_port_position_hash3'], line['end_port_position_hash4'], line['end_port_position_hash5']
#         first_hash3, first_hash4, first_hash5 = line['geo_hash3'].split(' ')[0], line['geo_hash4'].split(' ')[0], line['geo_hash5'].split(' ')[0]
#         last_hash3, last_hash4, last_hash5 = line['geo_hash3'].split(' ')[-1], line['geo_hash4'].split(' ')[-1], line['geo_hash5'].split(' ')[-1]
#         first_year, first_month, first_day = timestamp[0].year, timestamp[0].month, timestamp[0].day
#         last_year, last_month, last_day = timestamp[-1].year, timestamp[-1].month, timestamp[-1].day

#         # 距离特征
#         begin_distance = [distance(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)]  # 与起始距离
#         begin_dis_cumsum = np.cumsum(begin_distance)   # 累积距离
#         begin_dis_diff1 = np.diff(begin_distance, prepend=begin_distance[0])   # 一阶差分距离

#         end_distance = [distance(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]    # 与终止距离
#         end_dis_cumsum = np.cumsum(end_distance)    # 累积距离
#         end_dis_diff1 = np.diff(end_distance, prepend=end_distance[0]) 

#         begin_haversine = [haversine(begin_port[0], begin_port[1], lon[i], lat[i]) for i in range(xlen)] 
#         begin_hav_cumsum = np.cumsum(begin_haversine)   # 累积距离
#         begin_hav_diff1 = np.diff(begin_haversine, prepend=begin_haversine[0])   # 一阶差分距离

#         end_haversine = [haversine(end_port[0], end_port[1], lon[i], lat[i]) for i in range(xlen)]
#         end_hav_cumsum = np.cumsum(end_haversine)   # 累积距离
#         end_hav_diff1 = np.diff(end_haversine, prepend=end_haversine[0])   # 一阶差分距离

#         # 速度特征
#         timestamp_diff1_tmp = timestamp_diff1.copy()
#         timestamp_diff1_tmp[timestamp_diff1_tmp==0] = np.mean(timestamp_diff1_tmp)
#         diff_speed = begin_hav_diff1 / (timestamp_diff1_tmp + 1e-8)    # 差分路程 / 差分时间 = 速度1
        
#         timestamp_hours_tmp = timestamp_hours.copy()
#         timestamp_hours_tmp[timestamp_hours_tmp == 0] = np.mean(timestamp_hours_tmp)
#         cumsum_speed = begin_hav_cumsum / (timestamp_hours_tmp + 1e-8)  # 累积路程 / 累积时间 = 速度2

#         # 简单数值特征
#         speed_0_nums = (np.array(speed) == 0).sum()
#         speed_0_rate = speed_0_nums / xlen 
#         speed_0_time = timestamp_diff1[np.array(speed) == 0].sum()
#         hash3_nunique = len(set(line['geo_hash3'].split(' ')))
#         hash4_nunique = len(set(line['geo_hash4'].split(' ')))
#         hash5_nunique = len(set(line['geo_hash5'].split(' ')))
#         hash3_rate, hash4_rate, hash5_rate = hash3_nunique/xlen,  hash4_nunique/xlen, hash5_nunique/xlen 
#         first_begin_dis, first_begin_hav = distance(begin_port[0], begin_port[1], lon[0], lat[0]), haversine(begin_port[0], begin_port[1], lon[0], lat[0])
#         last_begin_dis, last_begin_hav = distance(begin_port[0], begin_port[1], lon[-1], lat[-1]), haversine(begin_port[0], begin_port[1], lon[-1], lat[-1])
#         first_end_dis, first_end_hav = distance(end_port[0], end_port[1], lon[0], lat[0]), haversine(end_port[0], end_port[1], lon[0], lat[0])
#         last_end_dis, last_end_hav = distance(end_port[0], end_port[1], lon[-1], lat[-1]), haversine(end_port[0], end_port[1], lon[-1], lat[-1])
#         fbd_lbd, fbd_lbd_rate = last_begin_dis/first_begin_dis, last_begin_dis-first_begin_dis
#         fbh_lbh, fbh_lbh_rate = last_begin_hav/first_begin_hav, last_begin_hav-first_begin_hav
#         fed_led, fed_led_rate = first_end_dis/last_end_dis, first_end_dis-last_end_dis
#         fed_leh, fed_leh_rate = first_end_hav/last_end_hav, first_end_hav-last_end_hav


#         # 正式添加统计特征
#         num_feas1 = get_stats(timestamp_hours) + get_stats(timestamp_diff1) + get_stats(lon) + get_stats(lat) + get_stats(speed) + get_stats(direction)
#         num_feas2 = get_stats(begin_distance)+get_stats(begin_dis_cumsum)+get_stats(begin_dis_diff1)+get_stats(end_distance)+get_stats(end_dis_cumsum)+get_stats(end_dis_diff1)
#         num_feas3 = get_stats(begin_haversine)+get_stats(begin_hav_cumsum)+get_stats(begin_hav_diff1)+get_stats(end_haversine)+get_stats(end_hav_cumsum)+get_stats(end_hav_diff1)
#         num_feas4 = get_stats(diff_speed) + get_stats(cumsum_speed)
#         num_feas5 = get_stats(lon_diff1) + get_stats(lat_diff1) + get_stats(speed_diff1) + get_stats(direction_diff1)
#         num_feas6 = [xlen, speed_0_nums, speed_0_rate, speed_0_time, hash3_nunique, hash4_nunique, hash5_nunique, 
#                      hash3_rate, hash4_rate, hash5_rate, first_begin_dis, first_begin_hav,  last_begin_dis, last_begin_hav,
#                      first_end_dis, first_end_hav, last_end_dis, last_end_hav, fbd_lbd, fbd_lbd_rate, 
#                      fbh_lbh, fbh_lbh_rate, fed_led, fed_led_rate, fed_leh, fed_leh_rate]
#         # print(len(num_feas1+num_feas2+num_feas3))
#         # break 

#         # 添加类别型特征
#         cat_feas1 = [carrierName, vesselMMSI, begin_hash3, begin_hash4, begin_hash5, 
#                      end_hash3, end_hash4, end_hash5, first_hash3, first_hash4, first_hash5, 
#                      last_hash3, last_hash4, last_hash5, first_year, first_month, first_day, 
#                      last_year, last_month, last_day]
        
#         numerical_fea.append(num_feas1 + num_feas2 + num_feas3 + num_feas4 + num_feas5 + num_feas6)
#         categorical_fea.append(cat_feas1)
#         if is_train:
#             label.append(line['label'])
    
#     numerical_fea = np.array(numerical_fea)
#     categorical_fea = np.array(categorical_fea)
#     # print(numerical_fea.shape)
#     num_cols = ['num_{}'.format(str(i)) for i in range(numerical_fea.shape[1])]
#     cat_cols = ['cat_{}'.format(str(i)) for i in range(categorical_fea.shape[1])]
#     num_df = pd.DataFrame(numerical_fea)
#     print(num_df.shape)
#     cat_df = pd.DataFrame(categorical_fea)
#     print(cat_df.shape)
#     fea_df = pd.concat([num_df, cat_df], axis=1)
    
#     fea_df.columns = num_cols + cat_cols
#     if is_train:
#         fea_df['label'] = label
#     print(fea_df.shape)
#     return fea_df

# test_fea = feature_engineering(test_df, is_train=False)
# train_fea = feature_engineering(train_df)

  0%|          | 6/155644 [00:00<47:05, 55.09it/s]

(228, 362)
(228, 20)
(228, 382)


100%|██████████| 155644/155644 [1:42:36<00:00, 25.28it/s]  


(155644, 362)
(155644, 20)
(155644, 383)
CPU times: user 1h 42min 56s, sys: 31.6 s, total: 1h 43min 28s
Wall time: 1h 42min 54s


In [9]:
# train_fea.to_pickle('data/train_fea_new.pkl')
# test_fea.to_pickle('data/test_fea_new.pkl')

# 读取特征文件

In [18]:
%%time

train_fea = pd.read_pickle("data/train_fea_new.pkl")
test_fea = pd.read_pickle("data/test_fea_round2.pkl")

CPU times: user 352 ms, sys: 420 ms, total: 772 ms
Wall time: 772 ms


In [19]:
print(train_fea.shape, test_fea.shape)
train_fea.head()

(155644, 383) (219, 382)


,num_0,num_1,num_2,num_3,num_4,num_5,num_6,num_7,num_8,num_9,num_10,num_11,num_12,num_13,num_14,num_15,num_16,num_17,num_18,num_19,num_20,num_21,num_22,num_23,num_24,num_25,num_26,num_27,num_28,num_29,num_30,num_31,num_32,num_33,num_34,num_35,num_36,num_37,num_38,num_39,num_40,num_41,num_42,num_43,num_44,num_45,num_46,num_47,num_48,num_49,num_50,num_51,num_52,num_53,num_54,num_55,num_56,num_57,num_58,num_59,num_60,num_61,num_62,num_63,num_64,num_65,num_66,num_67,num_68,num_69,num_70,num_71,num_72,num_73,num_74,num_75,num_76,num_77,num_78,num_79,num_80,num_81,num_82,num_83,num_84,num_85,num_86,num_87,num_88,num_89,num_90,num_91,num_92,num_93,num_94,num_95,num_96,num_97,num_98,num_99,num_100,num_101,num_102,num_103,num_104,num_105,num_106,num_107,num_108,num_109,num_110,num_111,num_112,num_113,num_114,num_115,num_116,num_117,num_118,num_119,num_120,num_121,num_122,num_123,num_124,num_125,num_126,num_127,num_128,num_129,num_130,num_131,num_132,num_133,num_134,num_135,num_136,num_137,num_138,num_139,num_140,num_141,num_142,num_143,num_144,num_145,num_146,num_147,num_148,num_149,num_150,num_151,num_152,num_153,num_154,num_155,num_156,num_157,num_158,num_159,num_160,num_161,num_162,num_163,num_164,num_165,num_166,num_167,num_168,num_169,num_170,num_171,num_172,num_173,num_174,num_175,num_176,num_177,num_178,num_179,num_180,num_181,num_182,num_183,num_184,num_185,num_186,num_187,num_188,num_189,num_190,num_191,num_192,num_193,num_194,num_195,num_196,num_197,num_198,num_199,num_200,num_201,num_202,num_203,num_204,num_205,num_206,num_207,num_208,num_209,num_210,num_211,num_212,num_213,num_214,num_215,num_216,num_217,num_218,num_219,num_220,num_221,num_222,num_223,num_224,num_225,num_226,num_227,num_228,num_229,num_230,num_231,num_232,num_233,num_234,num_235,num_236,num_237,num_238,num_239,num_240,num_241,num_242,num_243,num_244,num_245,num_246,num_247,num_248,num_249,num_250,num_251,num_252,num_253,num_254,num_255,num_256,num_257,num_258,num_259,num_260,num_261,num_262,num_263,num_264,num_265,num_266,num_267,num_268,num_269,num_270,num_271,num_272,num_273,num_274,num_275,num_276,num_277,num_278,num_279,num_280,num_281,num_282,num_283,num_284,num_285,num_286,num_287,num_288,num_289,num_290,num_291,num_292,num_293,num_294,num_295,num_296,num_297,num_298,num_299,num_300,num_301,num_302,num_303,num_304,num_305,num_306,num_307,num_308,num_309,num_310,num_311,num_312,num_313,num_314,num_315,num_316,num_317,num_318,num_319,num_320,num_321,num_322,num_323,num_324,num_325,num_326,num_327,num_328,num_329,num_330,num_331,num_332,num_333,num_334,num_335,num_336,num_337,num_338,num_339,num_340,num_341,num_342,num_343,num_344,num_345,num_346,num_347,num_348,num_349,num_350,num_351,num_352,num_353,num_354,num_355,num_356,num_357,num_358,num_359,num_360,num_361,cat_0,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,label
0,56.431096,102.866389,0.0,45.919525,102.866389,2087.950556,86.045833,36.0,37.0,0.972973,1.0,-0.247816,-1.859528,0.0,2.780173,33.933056,0.0,7.038709,33.933056,102.866389,0.312222,36.0,37.0,0.972973,1.000000,3.209655,9.766005,0.000000,108.741768,114.403907,103.757892,4.758537,10.646015,4023.445417,105.066755,37.0,37.0,1.0,1.0,0.200467,-1.902216,114.119610,10.423120,22.331625,1.151323,9.743013,21.180302,385.655431,2.531277,37.0,37.0,1.0,1.0,0.259001,-1.873041,22.331625,18.648649,30.0,0.0,9.349711,30.0,690.0,22.0,32.0,15.0,0.864865,0.405405,-1.040181,-0.210720,24.0,17354.486486,25610.0,-1.0,7879.100578,25611.0,642116.0,20610.0,37.0,34.0,1.0,0.918919,-1.037633,-0.061277,-1.0,13.197900,23.667174,0.090549,10.791027,23.576625,488.322318,21.860385,37.0,37.0,1.0,1.0,-0.248973,-1.878757,0.090549,146.637121,488.322318,0.090549,163.124255,488.231769,5425.573492,73.563311,37.0,37.0,1.0,1.0,0.715951,-0.951381,0.090549,0.637206,8.504889,0.000000,1.853925,8.504889,23.576625,0.051390,36.0,37.0,0.972973,1.0,3.528173,11.241187,0.0,87.753126,98.171112,79.048360,8.738200,19.122752,

In [20]:
use_cols = []
# imp = pd.read_pickle("imp_2.pkl")
# del_cols = imp['feat'].values.tolist()[-30:]

for col in ['num_{}'.format(str(i)) for i in range(362)]:
    if np.std(train_fea[col].values) > 0.01:
        use_cols.append(col)
print(len(use_cols))

339


In [21]:
# label_encoder
from sklearn.preprocessing import LabelEncoder

test_fea['label'] = -1e8
data_fea = pd.concat([train_fea, test_fea])
data_fea = data_fea.reset_index(drop=True)

for col in ['cat_{}'.format(str(i)) for i in range(20)]:
    print(col)
    le = LabelEncoder()
    data_fea[col] = le.fit_transform(data_fea[col])
    data_fea[col] = data_fea[col].astype('category')

train_feas = data_fea[data_fea['label'] != -1e8].reset_index(drop=True)
test_feas = data_fea[data_fea['label'] == -1e8].reset_index(drop=True)
del test_feas['label']
print(train_feas.shape, test_feas.shape)

all_feas = use_cols + ['cat_{}'.format(str(i)) for i in range(20)]
cat_feas = ['cat_{}'.format(str(i)) for i in range(20)]
print(len(all_feas), len(cat_feas))

cat_0
cat_1
cat_2
cat_3
cat_4
cat_5
cat_6
cat_7
cat_8
cat_9
cat_10
cat_11
cat_12
cat_13
cat_14
cat_15
cat_16
cat_17
cat_18
cat_19
(155644, 383) (219, 382)
359 20


In [11]:
%%time


file_name = datetime.date.today().strftime('%m%d')+"_{}.log".format("lgb_round2_origin")
def write_log(w):
    t0 = datetime.datetime.now().strftime('%H:%M:%S')
    info = "{} : {}\n".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info)
        f.write("-"*80+"\n")

def mse_score_eval(preds, valid):
    labels = valid.get_label()
    scores = mse(y_true=labels, y_pred=preds)
    return 'mse_score', scores, False

def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = all_feas

    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 10
    # Kfold
    # fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    # groups = train_df['loadingOrder'].values
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.01,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 128,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.75,
        'bagging_freq': 5,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': 24,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        write_log("fold {}".format(n_fold))
        train_x, train_y = train.iloc[train_idx], label[train_idx]
        valid_x, valid_y = train.iloc[valid_idx], label[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

        clf = lgb.train(
            params=params,
            train_set=n_train,
            categorical_feature=cat_feas,
            num_boost_round=10000,
            valid_sets=[n_train, n_valid],
            early_stopping_rounds=50,
            verbose_eval=100,
            feval=mse_score_eval
        )
        clf.save_model("data/lgb_save_model/round1_fold_{}.txt".format(n_fold))
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        write_log("val_mse = {}".format(mse(valid_y, train_pred[valid_idx])))
        
        test_pred += clf.predict(test, num_iteration=clf.best_iteration)/fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')

    write_log("train mse: {}".format(mse(label, train_pred)))
    result = pd.DataFrame({
        'loadingOrder': test_df['loadingOrder'], 
        'label': test_pred,
    })
    return result, imp

result, imp = build_model(train_feas[all_feas], test_feas[all_feas], train_feas['label'].values, is_shuffle=True)

result.to_pickle("result/result_round2_origin.pkl")
imp.to_pickle("result/fea1_round2_origin.pkl")

07:49:11 : fold 1



/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 6885.1	training's mse_score: 6885.1	valid_1's l2: 6858.12	valid_1's mse_score: 6858.12
[200]	training's l2: 1691.4	training's mse_score: 1691.4	valid_1's l2: 1852.86	valid_1's mse_score: 1852.86
[300]	training's l2: 734.603	training's mse_score: 734.603	valid_1's l2: 939.447	valid_1's mse_score: 939.447
[400]	training's l2: 461.794	training's mse_score: 461.794	valid_1's l2: 690.397	valid_1's mse_score: 690.397
[500]	training's l2: 323.085	training's mse_score: 323.085	valid_1's l2: 561.835	valid_1's mse_score: 561.835
[600]	training's l2: 243.717	training's mse_score: 243.717	valid_1's l2: 482.112	valid_1's mse_score: 482.112
[700]	training's l2: 193.383	training's mse_score: 193.383	valid_1's l2: 434.272	valid_1's mse_score: 434.272
[800]	training's l2: 158.704	training's mse_score: 158.704	valid_1's l2: 398.592	valid_1's mse_score: 398.592
[900]	training's l2: 133.223	training's mse_score: 133.223	vali

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 6876.64	training's mse_score: 6876.64	valid_1's l2: 6989.98	valid_1's mse_score: 6989.98
[200]	training's l2: 1685	training's mse_score: 1685	valid_1's l2: 1883.32	valid_1's mse_score: 1883.32
[300]	training's l2: 728.735	training's mse_score: 728.735	valid_1's l2: 954.534	valid_1's mse_score: 954.534
[400]	training's l2: 457.761	training's mse_score: 457.761	valid_1's l2: 692.508	valid_1's mse_score: 692.508
[500]	training's l2: 320.674	training's mse_score: 320.674	valid_1's l2: 555.023	valid_1's mse_score: 555.023
[600]	training's l2: 241.475	training's mse_score: 241.475	valid_1's l2: 477.322	valid_1's mse_score: 477.322
[700]	training's l2: 190.953	training's mse_score: 190.953	valid_1's l2: 430.588	valid_1's mse_score: 430.588
[800]	training's l2: 156.975	training's mse_score: 156.975	valid_1's l2: 399.892	valid_1's mse_score: 399.892
[900]	training's l2: 131.833	training's mse_score: 131.833	valid_

...

Training until validation scores don't improve for 50 rounds
[100]	training's l2: 6871.13	training's mse_score: 6871.13	valid_1's l2: 6850.75	valid_1's mse_score: 6850.75
[200]	training's l2: 1682.14	training's mse_score: 1682.14	valid_1's l2: 1808.5	valid_1's mse_score: 1808.5
[300]	training's l2: 730.057	training's mse_score: 730.057	valid_1's l2: 916.522	valid_1's mse_score: 916.522
[400]	training's l2: 458.083	training's mse_score: 458.083	valid_1's l2: 678.478	valid_1's mse_score: 678.478
[500]	training's l2: 321.201	training's mse_score: 321.201	valid_1's l2: 557.645	valid_1's mse_score: 557.645
[600]	training's l2: 242.576	training's mse_score: 242.576	valid_1's l2: 486.317	valid_1's mse_score: 486.317
[700]	training's l2: 191.963	training's mse_score: 191.963	valid_1's l2: 439.051	valid_1's mse_score: 439.051
[800]	training's l2: 158.152	training's mse_score: 158.152	valid_1's l2: 408.158	valid_1's mse_score: 408.158
[900]	training's l2: 132.967	training's mse_score: 132.967	va

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 6872.38	training's mse_score: 6872.38	valid_1's l2: 6933.8	valid_1's mse_score: 6933.8
[200]	training's l2: 1683.9	training's mse_score: 1683.9	valid_1's l2: 1817.43	valid_1's mse_score: 1817.43
[300]	training's l2: 728.756	training's mse_score: 728.756	valid_1's l2: 923.678	valid_1's mse_score: 923.678
[400]	training's l2: 459.193	training's mse_score: 459.193	valid_1's l2: 688.234	valid_1's mse_score: 688.234
[500]	training's l2: 321.759	training's mse_score: 321.759	valid_1's l2: 563.051	valid_1's mse_score: 563.051
[600]	training's l2: 242.301	training's mse_score: 242.301	valid_1's l2: 489.445	valid_1's mse_score: 489.445
[700]	training's l2: 191.285	training's mse_score: 191.285	valid_1's l2: 440.586	valid_1's mse_score: 440.586
[800]	training's l2: 157.007	training's mse_score: 157.007	valid_1's l2: 406.162	valid_1's mse_score: 406.162
[900]	training's l2: 132.314	training's mse_score: 132.314	vali

/opt/conda/envs/python3/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_0', 'cat_1', 'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16', 'cat_17', 'cat_18', 'cat_19', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 50 rounds
[100]	training's l2: 6848.99	training's mse_score: 6848.99	valid_1's l2: 7145.26	valid_1's mse_score: 7145.26
[200]	training's l2: 1682.88	training's mse_score: 1682.88	valid_1's l2: 1978.5	valid_1's mse_score: 1978.5
[300]	training's l2: 731.438	training's mse_score: 731.438	valid_1's l2: 1038.31	valid_1's mse_score: 1038.31
[400]	training's l2: 458.692	training's mse_score: 458.692	valid_1's l2: 775.232	valid_1's mse_score: 775.232
[500]	training's l2: 321.402	training's mse_score: 321.402	valid_1's l2: 633.191	valid_1's mse_score: 633.191
[600]	training's l2: 242.963	training's mse_score: 242.963	valid_1's l2: 553.943	valid_1's mse_score: 553.943
[700]	training's l2: 192.805	training's mse_score: 192.805	valid_1's l2: 502.231	valid_1's mse_score: 502.231
[800]	training's l2: 159.218	training's mse_score: 159.218	valid_1's l2: 467.802	valid_1's mse_score: 467.802
[900]	training's l2: 133.706	training's mse_score: 133.706	va

# 根据保存的模型直接预测

In [30]:
def mse_score_eval(preds, valid):
    labels = valid.get_label()
    scores = mse(y_true=labels, y_pred=preds)
    return 'mse_score', scores, False

def build_model(train, test, label, seed=2020, is_shuffle=True):
    imp = pd.DataFrame()  # 特征重要性
    imp['feat'] = all_feas

    train_pred = np.zeros((train.shape[0], ))
    test_pred = np.zeros((test.shape[0], ))
    n_splits = 10
    # Kfold
    # fold = GroupKFold(n_splits=n_splits)
    fold = KFold(n_splits=n_splits, shuffle=is_shuffle, random_state=seed)
    # groups = train_df['loadingOrder'].values
    kf_way = fold.split(train)
    # params
    params = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 128,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.80,
        'bagging_freq': 5,
        'seed': 8,
        'bagging_seed': 1,
        'feature_fraction_seed': 7,
        'min_data_in_leaf': 20,
        'nthread': -1,
        'verbose': 1,
    }
    # train
    for n_fold, (train_idx, valid_idx) in enumerate(kf_way, start=1):
        print("fold ", n_fold)
        train_x, train_y = train.iloc[train_idx], label[train_idx]
        valid_x, valid_y = train.iloc[valid_idx], label[valid_idx]
        # 数据加载
        n_train = lgb.Dataset(train_x, label=train_y)
        n_valid = lgb.Dataset(valid_x, label=valid_y)

#         clf = lgb.train(
#             params=params,
#             train_set=n_train,
#             categorical_feature=cat_feas,
#             num_boost_round=10000,
#             valid_sets=[n_train, n_valid],
#             early_stopping_rounds=50,
#             verbose_eval=100,
#             feval=mse_score_eval
#         )
#         clf.save_model("data/lgb_model_fold_{}.txt".format(n_fold))
        clf = lgb.Booster(model_file="data/lgb_model_fold_{}.txt".format(n_fold))
        train_pred[valid_idx] = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_pred += clf.predict(test, num_iteration=clf.best_iteration)/fold.n_splits

        imp['gain' + str(n_fold + 1)] = clf.feature_importance(importance_type='gain')
        imp['split' + str(n_fold + 1)] = clf.feature_importance(importance_type='split')

    print("train mse: ", mse(label, train_pred))
    result = pd.DataFrame({
        'loadingOrder': test_df['loadingOrder'], 
        'label': test_pred,
    })
    return result, imp

result, imp = build_model(train_feas[all_feas], test_feas[all_feas], train_feas['label'].values, is_shuffle=True)

fold  1
fold  2
fold  3
fold  4
fold  5
fold  6
fold  7
fold  8
fold  9
fold  10
train mse:  256.2317758079061


In [22]:
result

,loadingOrder,label
0,AC437723355280,412.804033
1,AC860038925693,433.040137
2,AR701843002140,111.349342
3,AT407433613767,369.079285
4,AV639724236766,237.980579
5,BA498843975994,120.771886
6,BB470439135271,643.635500
7,BE790761427541,426.552796
8,BE898272362291,523.156863
9,BE929753510083,515.772233


In [2]:
imp['gain'] = imp[[f for f in imp.columns if 'gain' in f]].sum(axis=1)/10
imp['split'] = imp[[f for f in imp.columns if 'split' in f]].sum(axis=1)/10

imp = imp[['feat', 'gain', 'split']]
imp = imp.sort_values(by=['gain'], ascending=False)
imp = imp.reset_index(drop=True)
# imp.to_pickle("imp_2.pkl")
imp

In [23]:
# train mse:  256.9597223109227
# train mse:  256.2317758079061

sub_FORMAT = "%Y/%m%d  %H:%M:%S"
UTC_FORMAT = "%Y-%m-%dT%H:%M:%S.%fZ"

def get_time(x, y):
    x = x.split(' ')[-1]
    x = datetime.datetime.strptime(x, UTC_FORMAT)
    day = y // 24 
    seconds = int((y - (y//24)*24)*3600)
    res = x + datetime.timedelta(days=day, seconds=seconds)
    str_res = res.strftime(sub_FORMAT)
    return res 

# result = pd.DataFrame({
#         'loadingOrder': test_loadingOrder, 
#         'label': test_pred,
#     })

result['timestamp'] = test_df['timestamp']
result['ETA'] = list(map(lambda x, y: get_time(x, y), result['timestamp'], result['label']))

In [24]:
test_data = pd.read_csv("data/R2_ATest 0711.csv") 
def get_data(data, mode='train'):
    
    assert mode=='train' or mode=='test'
#     if mode=='train':
#         data['vesselNextportETA'] = pd.to_datetime(data['vesselNextportETA'], infer_datetime_format=True) # 转换时间
    if mode=='test':
        data['temp_timestamp'] = data['timestamp']
        data['onboardDate'] = pd.to_datetime(data['onboardDate'], infer_datetime_format=True)
    
    data['timestamp'] = pd.to_datetime(data['timestamp'], infer_datetime_format=True)
    return data

test_data = get_data(test_data, mode='test')

test_data.sort_values(['loadingOrder', 'timestamp'], inplace=True)
test_data = test_data.reset_index(drop=True) 

test_data = test_data.merge(result[['loadingOrder', 'ETA']], on='loadingOrder', how='left')
test_data['ETA'] = test_data['ETA'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data.drop(['direction','TRANSPORT_TRACE'],axis=1,inplace=True)
test_data['onboardDate'] = test_data['onboardDate'].apply(lambda x:x.strftime('%Y/%m/%d  %H:%M:%S'))
test_data['creatDate'] = pd.datetime.now().strftime('%Y/%m/%d  %H:%M:%S')
test_data['timestamp'] = test_data['temp_timestamp']
# 整理columns顺序
result = test_data[['loadingOrder', 'timestamp', 'longitude', 'latitude', 'carrierName', 'vesselMMSI', 'onboardDate', 'ETA', 'creatDate']]

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:23: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.


In [28]:
result

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,AC437723355280,2019-06-23T02:53:18.000Z,113.895823,22.383463,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
1,AC437723355280,2019-06-23T03:01:28.000Z,113.917878,22.360128,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
2,AC437723355280,2019-06-23T03:16:38.000Z,113.977712,22.345253,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
3,AC437723355280,2019-06-23T03:17:08.000Z,113.980273,22.345310,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
4,AC437723355280,2019-06-23T03:18:08.000Z,113.984088,22.345340,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
...,...,...,...,...,...,...,...,...,...
77649,ZZ318732594946,2020-03-19T15:11:08.000Z,120.970747,25.253573,OIEQNT,C7827726749,2020/03/17 16:51:38,2020/04/09 18:08:56,2020/07/13 13:31:38
77650,ZZ318732594946,2020-03-19T15:11:58.000Z,120.973587,25.256897,OIEQNT,C7827726749,2020/03/17 16:51:38,2020/04/09 18:08:56,2020/07/13 13:31:38
77651,ZZ318732594946,2020-03-19T15:11:58.000Z,120.973587,25.256897,OIEQNT,C7827726749,2020/03/17 16:51:38,2020/04/09 18:08:56,2020/07/13 13:31:38
77652,ZZ318732594946,2020-03-19T15:15:38.000Z,120.986673,25.272215,OIEQNT,C7827726749,2020/03/17 16:51:38,2020/04/09 18:08:56,2020/07/13 13:31:38


In [27]:
tmp = result.drop_duplicates('loadingOrder').reset_index(drop=True)
tmp 

,loadingOrder,timestamp,longitude,latitude,carrierName,vesselMMSI,onboardDate,ETA,creatDate
0,AC437723355280,2019-06-23T02:53:18.000Z,113.895823,22.383463,JCMFTA,G9916514058,2019/06/23 02:53:18,2019/07/20 19:05:42,2020/07/13 13:31:38
1,AC860038925693,2019-06-17T13:24:48.000Z,114.290705,22.564435,JCMFTA,M1075899155,2019/06/17 13:24:48,2019/07/10 20:53:02,2020/07/13 13:31:38
2,AR701843002140,2020-03-25T15:55:58.000Z,113.694225,22.630953,OYSCFP,O8358265987,2020/03/25 15:49:58,2020/03/30 07:50:15,2020/07/13 13:31:38
3,AT407433613767,2019-04-01T00:14:18.000Z,104.196312,1.287627,RWHZVZ,J3697709469,2019/04/01 00:14:18,2019/04/26 05:25:03,2020/07/13 13:31:38
4,AV639724236766,2020-06-01T19:13:22.000Z,113.894683,22.451200,OYSCFP,R7006912480,2020/06/01 19:13:22,2020/06/11 21:57:10,2020/07/13 13:31:38
5,BA498843975994,2020-06-16T02:23:12.000Z,113.899117,22.461683,JONOCD,U9615709922,2020/06/16 02:23:12,2020/06/22 00:44:16,2020/07/13 13:31:38
6,BB470439135271,2020-04-28T00:00:19.000Z,129.496267,34.140347,OIEQNT,Q2654292540,2020/04/28 00:00:19,2020/05/25 19:28:41,2020/07/13 13:31:38
7,BE790761427541,2020-06-10T13:59:56.000Z,113.883267,22.444100,OYSCFP,I9946011301,2020/06/10 13:59:56,2020/06/28 18:33:09,2020/07/13 13:31:38
8,BE898272362291,2019-11-19T05:07:38.000Z,114.273142,22.563410,OYSCFP,C2075927370,2019/11/19 05:07:38,2019/12/12 17:01:12,2020/07/13 13:31:38
9,BE929753510083,2019-07-06T23:04:08.000Z,114.285403,22.576393,OYSCFP,B8183659437,2019/07/06 23:04:08,2019/07/31 05:04:38,2020/07/13 13:31:38


In [29]:
result.to_csv('result/round1_model.csv', index=False)

In [12]:
result

,loadingOrder,label
0,AC437723355280,412.804033
1,AC860038925693,433.040137
2,AR701843002140,111.349342
3,AT407433613767,369.079285
4,AV639724236766,237.980579
5,BA498843975994,120.771886
6,BB470439135271,643.635500
7,BE790761427541,426.552796
8,BE898272362291,523.156863
9,BE929753510083,515.772233


# 分析一下结果

In [6]:
import pandas as pd
res1 = pd.read_csv("data/B_lgb_10fold_old_add_fea.csv")
res2 = pd.read_csv("data/B_lgb_10fold_old.csv")
res1.drop_duplicates('loadingOrder', inplace=True)
res2.drop_duplicates('loadingOrder', inplace=True)

res1['onboardDate'] = pd.to_datetime(res1['onboardDate'], infer_datetime_format=True)
res1['ETA'] =  pd.to_datetime(res1['ETA'], infer_datetime_format=True)
res2['onboardDate'] = pd.to_datetime(res2['onboardDate'], infer_datetime_format=True)
res2['ETA'] =  pd.to_datetime(res2['ETA'], infer_datetime_format=True)

res1['diff'] = (res1['ETA'] - res1['onboardDate']).dt.total_seconds()
res2['diff'] = (res2['ETA'] - res2['onboardDate']).dt.total_seconds()

In [7]:
res1['diff'].corr(res2['diff'])

0.9994296946716894